<a href="https://colab.research.google.com/github/EbrahimZarei64/CNNHSIE/blob/main/VGG16.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torchvision.transforms as transforms
from PIL import Image
import torchvision.models as models
import time
import requests
from io import BytesIO

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)

vgg16.classifier = torch.nn.Sequential(
    vgg16.classifier[0],
    vgg16.classifier[1],
    vgg16.classifier[3],
    vgg16.classifier[4]
)
vgg16.classifier[0] = torch.nn.Linear(25088, 2048)
vgg16.classifier[2] = torch.nn.Linear(2048, 512)
vgg16.classifier[1] = torch.nn.Sigmoid()
vgg16.classifier[3] = torch.nn.Sigmoid()

image_url = 'https:// The image URL'  #Example: 'https://sipi.usc.edu/database/download.php?vol=aerials&img=2.2.01'

# Download the image from the URL
response = requests.get(image_url)
image = Image.open(BytesIO(response.content)).convert('RGB')

# Preprocess the input image
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
input_tensor = transform(image).unsqueeze(0)

In [ ]:
# Get the avgpool flatten layer
# start = time.perf_counter()
avgpool_output = vgg16.features(input_tensor)
avgpool_output = vgg16.avgpool(avgpool_output)
classifier_output = vgg16.avgpool(avgpool_output)
flatten_output = torch.flatten(classifier_output, start_dim=1)
classifier_output = vgg16.to(device).classifier(flatten_output)

# Get the result of the desired layer
desired_output = classifier_output[:, :512]
output_array = torch.where(desired_output > 0.5, torch.ones_like(desired_output), torch.zeros_like(desired_output))
# print(time.perf_counter() - start)
print(output_array)